In [ ]:
from paz.backend.image import load_image, show_image, write_image
from paz.applications import HaarCascadeFrontalFace, MiniXceptionFER
import paz.processors as pr
from paz.backend.image.image import crop_image
from paz.backend.image import write_image
from paz.processors.detection import CropBoxes2D
import numpy as np
from pathlib import Path
from paz.abstract import SequentialProcessor
from retinaface import RetinaFace
from paz.abstract.messages import Box2D
from paz.backend import boxes
from paz.backend.image import opencv_image
import os

result = np.array([['0', '0']])
front_face_size = np.array([224, 224])
#loop through each of the c1 ... c9 folder
for distraction_num in range(0, 10):
    SideBody_folder_str = './datasets/DMD/SideBody/train/' + 'c%(number)01d/' % {"number": distraction_num}
    
    #loop through every sidebody frame in the folder cn   
    for file_name in os.listdir(SideBody_folder_str):
        if file_name.endswith(".png"):   #framename format: gxxsxsidebody_000xxx.png
            SideBody_frame_name = file_name
            SideBody_path = SideBody_folder_str + file_name
            
            #find its corresponding frontbody frame
            FrontBody_folder_str = './datasets/DMD/FrontBody/'
            FrontBody_frame_name = SideBody_frame_name[0:5] + 'front' + SideBody_frame_name[9:]
            FrontBody_path = FrontBody_folder_str + FrontBody_frame_name
            FrontBody_image = load_image(FrontBody_path)
            
            #Detect and crop the front face
            crop = CropBoxes2D()
            face = RetinaFace.detect_faces(FrontBody_path)
            coordinates = face['face_1']['facial_area']
            boxes2D = [Box2D(coordinates, score=0)]
            
            #edge case 1: PAZ fails to detect a face
            if (len(boxes2D) == 0):
                result = np.append(result, [[SideBody_path, 'noFaceDetected']], axis=0) # how to cover this case in test?
                print('No face detected for this frame')
                continue
        
            #edge case 2: PAZ detects multiple "faces" - some of these are non-face areas
            elif (len(boxes2D) >= 2):
                areas = np.array([])
                for box in boxes2D:
                    area = (box.coordinates[2]-box.coordinates[0]) * (box.coordinates[3]-box.coordinates[1])
                    areas = np.append(areas, area)
                indices_excluding_max = np.where(areas != np.amax(areas))[0]
                for i in indices_excluding_max:
                    boxes2D = np.delete(boxes2D, i)
                print('After removing smaller boxes: ', boxes2D) 
            cropped_images = crop(FrontBody_image, boxes2D)
            
            #Label its emotion
            classify = MiniXceptionFER()
            emotion = classify(cropped_images[0])['class_name']

            #Add a row to emo_list [original SideBody frame path, emotion]
            result = np.append(result, [[SideBody_path, emotion]], axis=0) 
            
            #Resize the cropped face to 224*224
            if (cropped_images[0].shape[0] != cropped_images[0].shape[1]):
                new_coordinates = boxes.make_box_square(boxes2D[0].coordinates)
                box2D = Box2D(new_coordinates, 0)
                boxes2D = [box2D]
                cropped_images = crop(FrontBody_image, boxes2D)
            resized_image = opencv_image.resize_image(cropped_images[0], front_face_size)
                    
            #create directories to store the cropped faces
            os.makedirs('./pseudo_emotion_label/DMD/imgs/', exist_ok=True)
            
            #Save the cropped front face images
            cropped_image_name = FrontBody_frame_name[0:5] + 'frontcropped' + FrontBody_frame_name[14:]
            save_path = './pseudo_emotion_label/DMD/imgs/' + cropped_image_name
            write_image(save_path, resized_image)
    
        #skip non-png files/directory, if any        
        else:
            continue
#write outputs to a .csv file after going through all the sidebody images in 10 cn folders
np.savetxt("emo_list.csv", result, delimiter=",", fmt='%s')
